In [7]:
ddimport torch
import torch.nn as nn
import numpy as np

In [3]:
def linear_anneal(t, T, start, final, percentage):
    ''' Linear annealing scheduler
    t: current timestep
    T: total timesteps
    start: initial value
    final: value after percentage*T steps
    percentage: percentage of T after which annealing finishes
    '''
    final_from_T = int(percentage * T)
    if t > final_from_T:
        return final
    else:
        return final + (start - final) * (final_from_T - t) / final_from_T

In [24]:
linear_anneal(17, 25, 1.0, 0.01, 0.8)

0.45999999999999996

In [17]:
xs = [1, 2, 3]
s = '_'.join(str(x) for x in xs)

In [18]:
s

'1_2_3'